In [11]:
import random, numpy as np
import matplotlib.pyplot as plt

from deap import base
from deap import creator
from deap import tools

In [10]:
no_of_generations = 1000 # decide, iterations

# decide, population size or no of individuals or solutions being considered in each generation
population_size = 300 

# chromosome (also called individual) in DEAP
# length of the individual or chrosome should be divisible by no. of variables 
# is a series of 0s and 1s in Genetic Algorithm

# here, one individual may be 
# [1,0,1,1,1,0,......,0,1,1,0,0,0,0] of length 100
# each element is called a gene or allele or simply a bit
# Individual in bit form is called a Genotype and is decoded to attain the Phenotype i.e. the 
size_of_individual = 100 

# above, higher the better but uses higher resources

# we are using bit flip as mutation,
# probability that a gene or allele will mutate or flip, 
# generally kept low, high means more random jumps or deviation from parents, which is generally not desired
probability_of_mutation = 0.05 

# no. of participants in Tournament selection
# to implement strategy to select parents which will mate to produce offspring
tournSel_k = 10 

# no, of variables which will vary,here we have x and y
# this is so because both variables are of same length and are represented by one individual
# here first 50 bits/genes represent x and the rest 50 represnt y.
no_of_variables = 2

bounds = [(-6,6),(-6,6)] # one tuple or pair of lower bound and upper bound for each variable
# same for both variables in our problem 

# CXPB  is the probability with which two individuals
#       are crossed or mated
# MUTPB is the probability for mutating an individual
CXPB, MUTPB = 0.5, 0.2

In [12]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))

# an Individual is a list with one more attribute called fitness
creator.create("Individual", list, fitness=creator.FitnessMin)

In [13]:
# instance of Toolbox class
toolbox = base.Toolbox()

# Attribute generator, generation function 
# toolbox.attr_bool(), when called, will draw a random integer between 0 and 1
# it is equivalent to random.randint(0,1)
toolbox.register("attr_bool", random.randint, 0, 1)

# here give the no. of bits in an individual i.e. size_of_individual, here 100
# depends upon decoding strategy, which uses precision
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, size_of_individual) 
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [14]:
def decode_all_x(individual,no_of_variables,bounds):
    '''
    returns list of decoded x in same order as we have in binary format in chromosome
    bounds should have upper and lower limit for each variable in same order as we have in binary format in chromosome 
    '''
    
    len_chromosome = len(individual)
    len_chromosome_one_var = int(len_chromosome/no_of_variables)
    bound_index = 0
    x = []
    
    # one loop each for x(first 50 bits of individual) and y(next 50 of individual)
    for i in range(0,len_chromosome,len_chromosome_one_var):
        # converts binary to decimial using 2**place_value
        chromosome_string = ''.join((str(xi) for xi in  individual[i:i+len_chromosome_one_var]))
        binary_to_decimal = int(chromosome_string,2)
        
        # this formula for decoding gives us two benefits
        # we are able to implement lower and upper bounds for each variable
        # gives us flexibility to choose chromosome of any length, 
        #      more the no. of bits for a variable, more precise the decoded value can be
        lb = bounds[bound_index][0]
        ub = bounds[bound_index][1]
        precision = (ub-lb)/((2**len_chromosome_one_var)-1)
        decoded = (binary_to_decimal*precision)+lb
        x.append(decoded)
        bound_index +=1
    
    # returns a list of solutions in phenotype o, here [x,y]
    return x

def objective_fxn(individual):     
    
    # decoding chromosome to get decoded x in a list
    x = decode_all_x(individual,no_of_variables,bounds)
    
    # the formulas to decode the chromosome of 0s and 1s to an actual number, the value of x or y
    decoded_x = x[0]
    decoded_y = x[1]

    # the himmelblau function
    # min ((x**2)+y-11)**2+(x+(y**2)-7)**2
    # objective function value for the decoded x and decoded y
    obj_function_value = ((decoded_x**2)+decoded_y-11)**2+(decoded_x+(decoded_y**2)-7)**2
    
    # the evaluation function needs to return an iterable with each element corresponding to 
    #     each element in weights provided while inheriting from base.fitness
    return [obj_function_value] 

In [16]:
def check_feasiblity(individual):
    '''
    Feasibility function for the individual. 
    Returns True if individual is feasible (or constraint not violated),
    False otherwise
    '''
    var_list = decode_all_x(individual,no_of_variables,bounds)
    if sum(var_list) < 0:
        return True
    else:
        return False


def penalty_fxn(individual):
    '''
    Penalty function to be implemented if individual is not feasible or violates constraint
    It is assumed that if the output of this function is added to the objective function fitness values,
    the individual has violated the constraint.
    '''
    var_list = decode_all_x(individual,no_of_variables,bounds)
    return sum(var_list)**2

In [17]:

# registering objetive function with constraint
toolbox.register("evaluate", objective_fxn) # privide the objective function here
toolbox.decorate("evaluate", tools.DeltaPenalty(check_feasiblity, 1000, penalty_fxn)) # constraint on our objective function

# registering basic processes using bulit in functions in DEAP
toolbox.register("mate", tools.cxTwoPoint) # strategy for crossover, this classic two point crossover
toolbox.register("mutate", tools.mutFlipBit, indpb=probability_of_mutation) # mutation strategy with probability of mutation
toolbox.register("select", tools.selTournament, tournsize=tournSel_k) # selection startegy

In [20]:
hall_of_fame = tools.HallOfFame(1)

In [21]:
stats = tools.Statistics()

# registering the functions to which we will pass the list of fitness's of a gneration's offspring
# to ge the results
stats.register('Min', np.min)
stats.register('Max', np.max)
stats.register('Avg', np.mean)
stats.register('Std', np.std)

logbook = tools.Logbook()

In [22]:
# create poppulation as coded in population class
# no. of individuals can be given as input
pop = toolbox.population(n=population_size)

# The next thing to do is to evaluate our brand new population.

# use map() from python to give each individual to evaluate and create a list of the result
fitnesses = list(map(toolbox.evaluate, pop)) 

# ind has individual and fit has fitness score
# individual class in deap has fitness.values attribute which is used to store fitness value
for ind, fit in zip(pop, fitnesses):
    ind.fitness.values = fit

# evolve our population until we reach the number of generations

# Variable keeping track of the number of generations
g = 0
# clearing hall_of_fame object as precaution before every run
hall_of_fame.clear()

# Begin the evolution
while g < no_of_generations:
    # A new generation
    g = g + 1
    
    #The evolution itself will be performed by selecting, mating, and mutating the individuals in our population.
    
    # the first step is to select the next generation.
    # Select the next generation individuals using select defined in toolbox here tournament selection
    # the fitness of populations is decided from the individual.fitness.values[0] attribute
    #      which we assigned earlier to each individual
    # these are best individuals selected from population after selection strategy
    offspring = toolbox.select(pop, len(pop))
    # Clone the selected individuals, this needs to be done to create copy and avoid problem of inplace operations
    # This is of utter importance since the genetic operators in toolbox will modify the provided objects in-place.
    offspring = list(map(toolbox.clone, offspring))
    
    # Next, we will perform both the crossover (mating) and the mutation of the produced children with 
    #        a certain probability of CXPB and MUTPB. 
    # The del statement will invalidate the fitness of the modified offspring as they are no more valid 
    #       as after crossover and mutation, the individual changes
    
    # Apply crossover and mutation on the offspring
    # note, that since we are not cloning, the changes in child1, child2 and mutant are happening inplace in offspring
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if random.random() < CXPB:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values 
                
                
    # Evaluate the individuals with an invalid fitness (after we use del to make them invalid)
    # again note, that since we did not use clone, each change happening is happening inplace in offspring
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit
        
    
    # To check the performance of the evolution, we will calculate and print the 
    # minimal, maximal, and mean values of the fitnesses of all individuals in our population 
    # as well as their standard deviations.
    # Gather all the fitnesses in one list and print the stats
    #this_gen_fitness = [ind.fitness.values[0] for ind in offspring]
    this_gen_fitness = [] # this list will have fitness value of all the offspring
    for ind in offspring:
        this_gen_fitness.append(ind.fitness.values[0])            
    
    
    #### SHORT METHOD
    
    # will update the HallOfFame object with the best individual 
    #   according to fitness value and weight (while creating base.Fitness class)
    hall_of_fame.update(offspring)
    
    # pass a list of fitnesses 
    # (basically an object on which we want to perform registered functions)
    # will return a dictionary with key = name of registered function and value is return of the registered function
    stats_of_this_gen = stats.compile(this_gen_fitness)
    
    # creating a key with generation number
    stats_of_this_gen['Generation'] = g
    
    # printing for each generation
    print(stats_of_this_gen)
    
    # recording everything in a logbook object
    # logbook is essentially a list of dictionaries
    logbook.append(stats_of_this_gen)
    
    
    # now one generation is over and we have offspring from that generation
    # these offspring wills serve as population for the next generation
    # this is not happening inplace because this is simple python list and not a deap framework syntax
    pop[:] = offspring

{'Min': 1.2192667980159009, 'Max': 1104.366663398794, 'Avg': 645.0404876131873, 'Std': 411.1419483351754, 'Generation': 1}
{'Min': 5.489796429087747, 'Max': 1124.1771833769235, 'Avg': 656.2444730091003, 'Std': 410.9103706252088, 'Generation': 2}
{'Min': 8.100670779051045, 'Max': 1172.4695544607653, 'Avg': 610.3939527437642, 'Std': 410.64326618698544, 'Generation': 3}
{'Min': 0.48357390111030235, 'Max': 1367.8541286307113, 'Avg': 662.7971134187445, 'Std': 411.42567093702326, 'Generation': 4}
{'Min': 2.2940827312799645, 'Max': 1112.8191174039057, 'Avg': 623.2785809147289, 'Std': 407.85571400510224, 'Generation': 5}
{'Min': 5.05474920102567, 'Max': 1482.572743866691, 'Avg': 667.459870806303, 'Std': 399.8492112620379, 'Generation': 6}
{'Min': 7.939701905770686, 'Max': 1112.2241970755865, 'Avg': 632.0627781509311, 'Std': 403.8906525541811, 'Generation': 7}
{'Min': 2.490533125229549, 'Max': 1549.0160027347024, 'Avg': 606.1441779047153, 'Std': 419.5301758966841, 'Generation': 8}
{'Min': 6.203

{'Min': 0.44980100324065103, 'Max': 1322.5551812635827, 'Avg': 657.5272207345406, 'Std': 403.88652635974086, 'Generation': 67}
{'Min': 3.344666946303446, 'Max': 1106.172044674886, 'Avg': 604.5225121767755, 'Std': 412.79332384132545, 'Generation': 68}
{'Min': 2.625681756104407, 'Max': 1105.2466620428709, 'Avg': 610.2967035331629, 'Std': 409.81981061418486, 'Generation': 69}
{'Min': 2.9696795629370527, 'Max': 1173.2369026508766, 'Avg': 664.4836145369518, 'Std': 405.8526267537944, 'Generation': 70}
{'Min': 0.5772206767693218, 'Max': 1254.1049889102194, 'Avg': 672.6579906889173, 'Std': 405.35728321258205, 'Generation': 71}
{'Min': 2.406662554907883, 'Max': 1222.7244999600832, 'Avg': 670.2819752201185, 'Std': 399.4906960772317, 'Generation': 72}
{'Min': 3.1497513269978246, 'Max': 1117.8490008096737, 'Avg': 624.1359613042567, 'Std': 413.44039902781105, 'Generation': 73}
{'Min': 5.149217310193046, 'Max': 1534.8739541354, 'Avg': 654.4245340615142, 'Std': 415.50078882749506, 'Generation': 74}
{

{'Min': 3.3080574579276183, 'Max': 1125.482481286898, 'Avg': 653.0454409466416, 'Std': 408.54377427380996, 'Generation': 133}
{'Min': 0.26261318426505903, 'Max': 1391.4673076406837, 'Avg': 622.2381135219439, 'Std': 412.37021482088835, 'Generation': 134}
{'Min': 1.5583449229765454, 'Max': 1122.0038252804675, 'Avg': 696.1997605059627, 'Std': 396.369982805811, 'Generation': 135}
{'Min': 2.7357259973055736, 'Max': 1113.9424777865395, 'Avg': 686.2221060264482, 'Std': 405.3874714911507, 'Generation': 136}
{'Min': 6.849980817948646, 'Max': 1124.091368139956, 'Avg': 635.3423353830981, 'Std': 404.20391918621664, 'Generation': 137}
{'Min': 5.329978690743527, 'Max': 1118.3399571814452, 'Avg': 671.8839740750623, 'Std': 406.2339171801545, 'Generation': 138}
{'Min': 7.268616726461948, 'Max': 1332.310192639782, 'Avg': 672.5888957953878, 'Std': 407.09820566250914, 'Generation': 139}
{'Min': 24.833804034455632, 'Max': 1218.7757947875884, 'Avg': 655.599021100144, 'Std': 404.5007126539101, 'Generation': 

{'Min': 4.9427812087077925, 'Max': 1345.9880848542987, 'Avg': 645.6396831531509, 'Std': 413.9601060299071, 'Generation': 199}
{'Min': 3.919451918125224, 'Max': 1125.877541880829, 'Avg': 671.7345894471077, 'Std': 408.81591761790764, 'Generation': 200}
{'Min': 4.5689223784657775, 'Max': 1127.373505527407, 'Avg': 670.6615446253263, 'Std': 406.6194737668949, 'Generation': 201}
{'Min': 0.6829203601983677, 'Max': 1126.6819700009682, 'Avg': 651.4466897866936, 'Std': 408.5864910228263, 'Generation': 202}
{'Min': 0.19340853656192086, 'Max': 1114.7688943479159, 'Avg': 639.0774471837066, 'Std': 411.2795427848326, 'Generation': 203}
{'Min': 0.5656749030508172, 'Max': 1417.9457343499619, 'Avg': 633.3264376059719, 'Std': 410.94924153695507, 'Generation': 204}
{'Min': 6.699213778821941, 'Max': 1133.160901048428, 'Avg': 648.4390261008709, 'Std': 406.6060392366214, 'Generation': 205}
{'Min': 8.870715991050469, 'Max': 1104.8179529942804, 'Avg': 633.1922700593241, 'Std': 417.98409574882845, 'Generation':

{'Min': 1.7161915769134377, 'Max': 1142.31860040651, 'Avg': 636.0312113227673, 'Std': 408.4321890687351, 'Generation': 266}
{'Min': 17.997992345524423, 'Max': 1136.3707333644697, 'Avg': 666.0003949346822, 'Std': 405.89572111527644, 'Generation': 267}
{'Min': 1.7057621307078656, 'Max': 1132.6870790358394, 'Avg': 653.5933483312778, 'Std': 413.70686882855455, 'Generation': 268}
{'Min': 2.0131903327907454, 'Max': 1106.31895344942, 'Avg': 645.4672387959955, 'Std': 416.3783372868879, 'Generation': 269}
{'Min': 16.88326044735455, 'Max': 1120.2505288354898, 'Avg': 647.8904717396307, 'Std': 406.1166623084165, 'Generation': 270}
{'Min': 4.856718222847582, 'Max': 1127.7858517669958, 'Avg': 639.7087936537351, 'Std': 413.9291808731757, 'Generation': 271}
{'Min': 1.207766676845145, 'Max': 1418.0276093736727, 'Avg': 670.9270150051899, 'Std': 401.63980188152823, 'Generation': 272}
{'Min': 3.9869253518534777, 'Max': 1213.2893531249838, 'Avg': 643.49027457833, 'Std': 404.5935064317461, 'Generation': 273

...

{'Min': 6.573973996089964, 'Max': 1115.5762393006153, 'Avg': 611.4602668530451, 'Std': 406.63833445034817, 'Generation': 728}
{'Min': 6.014300103948295, 'Max': 1115.0015169569174, 'Avg': 657.3948193556159, 'Std': 404.18317598755937, 'Generation': 729}
{'Min': 5.002927158834847, 'Max': 1422.322373007203, 'Avg': 597.8975852350496, 'Std': 409.6695095912035, 'Generation': 730}
{'Min': 1.069537256111364, 'Max': 1108.6545127847337, 'Avg': 634.3897019431304, 'Std': 410.59330116486524, 'Generation': 731}
{'Min': 18.71315429138463, 'Max': 1111.7688332800517, 'Avg': 627.6933207867811, 'Std': 415.852172092956, 'Generation': 732}
{'Min': 8.160421032396956, 'Max': 1126.8268647566488, 'Avg': 604.2207617776686, 'Std': 414.1556561547896, 'Generation': 733}
{'Min': 4.229743155852577, 'Max': 1141.9103063213577, 'Avg': 633.4611865919627, 'Std': 401.0751997340341, 'Generation': 734}
{'Min': 1.109150787433152, 'Max': 1127.557882413214, 'Avg': 607.3276772463032, 'Std': 416.4037271954879, 'Generation': 735}


{'Min': 5.892130995663969, 'Max': 1115.2412433177788, 'Avg': 626.0448149142757, 'Std': 415.0834862630873, 'Generation': 794}
{'Min': 3.616731355990605, 'Max': 1296.7681243305617, 'Avg': 645.7777286583681, 'Std': 415.84270197965753, 'Generation': 795}
{'Min': 7.973139749569196, 'Max': 1256.7686684205682, 'Avg': 652.0157657974523, 'Std': 405.76132530186806, 'Generation': 796}
{'Min': 1.4858949954714176, 'Max': 1243.7644676666027, 'Avg': 669.7558853636715, 'Std': 397.8377424194014, 'Generation': 797}
{'Min': 3.9222307746768976, 'Max': 1315.7835892088847, 'Avg': 653.0908056955691, 'Std': 415.38573522133305, 'Generation': 798}
{'Min': 1.8938835978262742, 'Max': 1120.843006653022, 'Avg': 630.7264537892295, 'Std': 397.92881132392534, 'Generation': 799}
{'Min': 7.345017551924182, 'Max': 1126.0310038161854, 'Avg': 666.8513774730249, 'Std': 411.84160956457754, 'Generation': 800}
{'Min': 1.134687678494255, 'Max': 1137.2022701542087, 'Avg': 647.0153201938543, 'Std': 406.4308584013029, 'Generation'

{'Min': 0.7445777746460476, 'Max': 1138.8326499235059, 'Avg': 632.3471701172767, 'Std': 406.9421930415508, 'Generation': 861}
{'Min': 3.224117418365566, 'Max': 1110.799836135221, 'Avg': 643.9243242757768, 'Std': 407.36422706696914, 'Generation': 862}
{'Min': 2.840303307721109, 'Max': 1119.1629542566861, 'Avg': 648.6295061155397, 'Std': 399.77716959850045, 'Generation': 863}
{'Min': 7.174876981828961, 'Max': 1150.5039797421393, 'Avg': 619.4864046510818, 'Std': 421.1377147073759, 'Generation': 864}
{'Min': 8.101312319880615, 'Max': 1124.8788378820561, 'Avg': 618.5484348318745, 'Std': 409.6968745978334, 'Generation': 865}
{'Min': 0.5160391096331687, 'Max': 1128.9418568026113, 'Avg': 674.1380892605192, 'Std': 401.70653289497585, 'Generation': 866}
{'Min': 2.495956082920644, 'Max': 1119.048417791927, 'Avg': 603.9257488681353, 'Std': 403.8057970482134, 'Generation': 867}
{'Min': 4.284609685086805, 'Max': 1273.3425413362502, 'Avg': 600.7597105347413, 'Std': 409.3841719003519, 'Generation': 86

{'Min': 0.7269500719311579, 'Max': 1135.7337116045383, 'Avg': 651.9843247423391, 'Std': 401.1090406604446, 'Generation': 927}
{'Min': 2.121699245538831, 'Max': 1353.4248517830551, 'Avg': 620.3394959979094, 'Std': 420.3799252101548, 'Generation': 928}
{'Min': 4.849332901311162, 'Max': 1133.1389998159577, 'Avg': 643.0692544248122, 'Std': 408.1313577504312, 'Generation': 929}
{'Min': 6.317500351055984, 'Max': 1125.7218716700643, 'Avg': 662.8973381782087, 'Std': 404.7887584206631, 'Generation': 930}
{'Min': 7.329443195356358, 'Max': 1204.8375649722348, 'Avg': 626.4264726582629, 'Std': 409.488232977064, 'Generation': 931}
{'Min': 3.6703525993376753, 'Max': 1127.597922527803, 'Avg': 652.3236900150885, 'Std': 405.97464310783295, 'Generation': 932}
{'Min': 6.449354380875528, 'Max': 1242.1614988946953, 'Avg': 665.3700348765993, 'Std': 409.18191836850957, 'Generation': 933}
{'Min': 0.8678368670969188, 'Max': 1312.9034854408546, 'Avg': 600.3855472500559, 'Std': 405.0593881046788, 'Generation': 93

{'Min': 2.257393944603008, 'Max': 1316.3094898501033, 'Avg': 608.1842115018951, 'Std': 426.0965583172501, 'Generation': 994}
{'Min': 19.5475809572499, 'Max': 1428.9004055438304, 'Avg': 627.0959054521342, 'Std': 408.83074651084974, 'Generation': 995}
{'Min': 12.632712595252075, 'Max': 1168.167179963637, 'Avg': 664.4399995791672, 'Std': 407.8265617319444, 'Generation': 996}
{'Min': 1.0043882905860317, 'Max': 1438.8467963996359, 'Avg': 621.9106512923695, 'Std': 430.9708071612763, 'Generation': 997}
{'Min': 12.2234839589042, 'Max': 1156.5656365490056, 'Avg': 624.2560731678608, 'Std': 413.2210023073364, 'Generation': 998}
{'Min': 2.426993079421027, 'Max': 1124.1894502450345, 'Avg': 650.8037677998823, 'Std': 403.2509245790674, 'Generation': 999}
{'Min': 0.07899077948242476, 'Max': 1296.5680222888423, 'Avg': 656.5690828826981, 'Std': 410.6085207860964, 'Generation': 1000}


In [23]:
# print the best solution using HallOfFame object
for best_indi in hall_of_fame:
    # using values to return the value and
    # not a deap.creator.FitnessMin object
    best_obj_val_overall = best_indi.fitness.values[0]
    print('Minimum value for function: ',best_obj_val_overall)
    print('Optimum Solution: ',decode_all_x(best_indi,no_of_variables,bounds))

Minimum value for function:  0.010799791586229414
Optimum Solution:  [-3.780624724605543, -3.267994280734262]


In [29]:
no_of_generations = 1000 # decide, iterations

population_size = 300 

size_of_individual = 100 

probability_of_mutation = 0.05 

tournSel_k = 10 

no_of_variables = 2

bounds = [(-2,2),(-2,2)] 

CXPB, MUTPB = 0.5, 0.2

creator.create("FitnessMin", base.Fitness, weights=(-1.0,))

creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()

toolbox.register("attr_bool", random.randint, 0, 1)

toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, size_of_individual) 

toolbox.register("population", tools.initRepeat, list, toolbox.individual)

def decode_all_x(individual,no_of_variables,bounds):

    len_chromosome = len(individual)
    len_chromosome_one_var = int(len_chromosome/no_of_variables)
    bound_index = 0
    x = []
    
    for i in range(0,len_chromosome,len_chromosome_one_var):
        
        chromosome_string = ''.join((str(xi) for xi in  individual[i:i+len_chromosome_one_var]))
        binary_to_decimal = int(chromosome_string,2)
        
        lb = bounds[bound_index][0]
        ub = bounds[bound_index][1]
        precision = (ub-lb)/((2**len_chromosome_one_var)-1)
        decoded = (binary_to_decimal*precision)+lb
        x.append(decoded)
        bound_index +=1

    return x

def objective_fxn(individual):     
    
    x = decode_all_x(individual,no_of_variables,bounds)
    
    decoded_x = x[0]
    decoded_y = x[1]

    obj_function_value = decoded_x+decoded_y
    
    return [obj_function_value] 

def check_feasiblity(individual):

    var_list = decode_all_x(individual,no_of_variables,bounds)
    if sum(var_list) < 0:
        return True
    else:
        return False

def penalty_fxn(individual):

    var_list = decode_all_x(individual,no_of_variables,bounds)
    return sum(var_list)**2

# registering objetive function with constraint
toolbox.register("evaluate", objective_fxn) # privide the objective function here
toolbox.decorate("evaluate", tools.DeltaPenalty(check_feasiblity, 1000, penalty_fxn)) # constraint on our objective function

# registering basic processes using bulit in functions in DEAP
toolbox.register("mate", tools.cxTwoPoint) # strategy for crossover, this classic two point crossover
toolbox.register("mutate", tools.mutFlipBit, indpb=probability_of_mutation) # mutation strategy with probability of mutation
toolbox.register("select", tools.selTournament, tournsize=tournSel_k) # selection startegy

hall_of_fame = tools.HallOfFame(1)

stats = tools.Statistics()

stats.register('Min', np.min)
stats.register('Max', np.max)
stats.register('Avg', np.mean)
stats.register('Std', np.std)

logbook = tools.Logbook()

pop = toolbox.population(n=population_size)

fitnesses = list(map(toolbox.evaluate, pop)) 

for ind, fit in zip(pop, fitnesses):
    ind.fitness.values = fit

g = 0

hall_of_fame.clear()

while g < no_of_generations:

    g = g + 1
    
    offspring = toolbox.select(pop, len(pop))

    offspring = list(map(toolbox.clone, offspring))

    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if random.random() < CXPB:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values 

    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    this_gen_fitness = [] # this list will have fitness value of all the offspring
    for ind in offspring:
        this_gen_fitness.append(ind.fitness.values[0])            

    hall_of_fame.update(offspring)

    stats_of_this_gen = stats.compile(this_gen_fitness)
    
    # creating a key with generation number
    stats_of_this_gen['Generation'] = g
    
    # printing for each generation
    print(stats_of_this_gen)

    logbook.append(stats_of_this_gen)

    pop[:] = offspring
    
# print the best solution using HallOfFame object
for best_indi in hall_of_fame:
    # using values to return the value and
    # not a deap.creator.FitnessMin object
    best_obj_val_overall = best_indi.fitness.values[0]
    print('Minimum value for function: ',best_obj_val_overall)
    print('Optimum Solution: ',decode_all_x(best_indi,no_of_variables,bounds))

{'Min': -3.62733198249278, 'Max': 1013.1807592542021, 'Avg': 460.43522278060817, 'Std': 500.2374185221414, 'Generation': 1}
{'Min': -3.7432212732121997, 'Max': 1015.1486686685638, 'Avg': 504.05357615044727, 'Std': 502.0312891774828, 'Generation': 2}
{'Min': -3.3853771802062798, 'Max': 1013.9244932584404, 'Avg': 493.85797982469404, 'Std': 501.78727749830335, 'Generation': 3}
{'Min': -3.6082249137376152, 'Max': 1013.4052653046991, 'Avg': 466.96346568378465, 'Std': 500.8080855581059, 'Generation': 4}
{'Min': -3.7025283083196, 'Max': 1014.7045417949461, 'Avg': 447.02288796496987, 'Std': 499.19450061985833, 'Generation': 5}
{'Min': -3.7844059392939364, 'Max': 1012.4467352218898, 'Avg': 490.37296820219365, 'Std': 501.59927930599076, 'Generation': 6}
{'Min': -3.71733129940451, 'Max': 1013.1016101412172, 'Avg': 420.1652265214409, 'Std': 495.422702700331, 'Generation': 7}
{'Min': -3.8183994993644887, 'Max': 1012.5886432273545, 'Avg': 487.1927937167522, 'Std': 501.68878940348196, 'Generation': 8

{'Min': -3.6751579934820455, 'Max': 1013.2919675391456, 'Avg': 423.67900613923905, 'Std': 496.0716695705418, 'Generation': 66}
{'Min': -3.732619186883216, 'Max': 1015.385407289576, 'Avg': 443.6435105238977, 'Std': 498.8292147128899, 'Generation': 67}
{'Min': -3.856145592051444, 'Max': 1014.4328051400156, 'Avg': 443.71360034582807, 'Std': 498.7782747177407, 'Generation': 68}
{'Min': -3.8465723062048696, 'Max': 1014.4290635335407, 'Avg': 450.26686225409446, 'Std': 499.3167620066984, 'Generation': 69}
{'Min': -3.907613115685983, 'Max': 1013.8013859512099, 'Avg': 483.8704714033774, 'Std': 501.6161186732036, 'Generation': 70}
{'Min': -3.751409930798623, 'Max': 1013.8891273666337, 'Avg': 477.27770199485326, 'Std': 501.6648989194403, 'Generation': 71}
{'Min': -3.8331011661387677, 'Max': 1014.0401685698356, 'Avg': 487.47808963823775, 'Std': 501.95612466996516, 'Generation': 72}
{'Min': -3.592611302251974, 'Max': 1013.8045053000506, 'Avg': 490.32722090284614, 'Std': 501.78001060935105, 'Generat

{'Min': -3.5808751342811216, 'Max': 1014.5893140551786, 'Avg': 500.5224887984809, 'Std': 501.97684657978965, 'Generation': 132}
{'Min': -3.68762788693882, 'Max': 1014.9073315646565, 'Avg': 447.1448663808772, 'Std': 499.30736640178117, 'Generation': 133}
{'Min': -3.8493216808577486, 'Max': 1012.0175181172929, 'Avg': 470.25596216617345, 'Std': 500.9809250514884, 'Generation': 134}
{'Min': -3.6949592211276183, 'Max': 1015.042476256553, 'Avg': 470.39318556694025, 'Std': 501.02385038661464, 'Generation': 135}
{'Min': -3.732109050157149, 'Max': 1013.9769741778161, 'Avg': 457.0625681395039, 'Std': 500.0468699996795, 'Generation': 136}
{'Min': -3.846545813308076, 'Max': 1014.1766815314645, 'Avg': 473.62725944799126, 'Std': 501.13793738011344, 'Generation': 137}
{'Min': -3.7328463724744836, 'Max': 1010.7640447589306, 'Avg': 473.6023426717776, 'Std': 500.98563065294525, 'Generation': 138}
{'Min': -3.6582285269399826, 'Max': 1012.9339113723576, 'Avg': 447.04711603611395, 'Std': 499.1328310958899,

{'Min': -3.7245010257006186, 'Max': 1013.6190049718422, 'Avg': 490.5312136102947, 'Std': 501.87434012585516, 'Generation': 197}
{'Min': -3.7710274475993906, 'Max': 1012.522990481731, 'Avg': 463.8060209226393, 'Std': 500.58748900234855, 'Generation': 198}
{'Min': -3.354435827037096, 'Max': 1014.0483493778844, 'Avg': 400.2380765203394, 'Std': 491.76959651580876, 'Generation': 199}
{'Min': -3.879083303658792, 'Max': 1013.0546842612353, 'Avg': 426.9371682763062, 'Std': 496.566425336165, 'Generation': 200}
{'Min': -3.9567423165286684, 'Max': 1015.2172102937096, 'Avg': 527.4169605317344, 'Std': 501.25354911101977, 'Generation': 201}
{'Min': -3.7614372367068896, 'Max': 1012.3638875887998, 'Avg': 440.2914810743693, 'Std': 498.24863686661433, 'Generation': 202}
{'Min': -3.89396152909017, 'Max': 1014.445013898362, 'Avg': 503.7917526363106, 'Std': 501.85343208689966, 'Generation': 203}
{'Min': -3.703616666418373, 'Max': 1012.5045542750269, 'Avg': 463.6857342677747, 'Std': 500.6184068406579, 'Gene

{'Min': -3.6016187513051956, 'Max': 1014.2429499123716, 'Avg': 463.70010398976535, 'Std': 500.60516713862273, 'Generation': 262}
{'Min': -3.8174575707955754, 'Max': 1014.8800916354063, 'Avg': 443.93170656700084, 'Std': 499.0177809019557, 'Generation': 263}
{'Min': -3.9765055775451437, 'Max': 1015.2241864802312, 'Avg': 483.7653141166829, 'Std': 501.757076548084, 'Generation': 264}
{'Min': -3.74575412423086, 'Max': 1014.7196837745448, 'Avg': 480.5799905254079, 'Std': 501.6273765906042, 'Generation': 265}
{'Min': -3.906644906240423, 'Max': 1013.4101695081264, 'Avg': 497.24410467336577, 'Std': 501.8815959361744, 'Generation': 266}
{'Min': -3.883411266300154, 'Max': 1015.2038879448635, 'Avg': 457.075793913077, 'Std': 500.13936588644884, 'Generation': 267}
{'Min': -3.6599451615873377, 'Max': 1015.0455211270331, 'Avg': 416.85772121090224, 'Std': 494.96362211992164, 'Generation': 268}
{'Min': -3.70374721875233, 'Max': 1012.9750708335218, 'Avg': 457.07155288695844, 'Std': 500.084862384035, 'Gen

...

{'Min': -3.7611760746508445, 'Max': 1013.8590212067244, 'Avg': 463.90130764802007, 'Std': 500.68353060707284, 'Generation': 719}
{'Min': -3.746380887718331, 'Max': 1014.6463794032176, 'Avg': 500.32133500730174, 'Std': 501.8442574124136, 'Generation': 720}
{'Min': -3.6931122180460783, 'Max': 1013.8766992268615, 'Avg': 477.28795753827677, 'Std': 501.60514282826153, 'Generation': 721}
{'Min': -3.6808426728569437, 'Max': 1013.384917478095, 'Avg': 397.1050878988097, 'Std': 491.2412368826832, 'Generation': 722}
{'Min': -3.794831337832953, 'Max': 1013.5329679532595, 'Avg': 527.5749795507439, 'Std': 501.53550706482764, 'Generation': 723}
{'Min': -3.6919167757539064, 'Max': 1014.2982801369743, 'Avg': 537.3458178310349, 'Std': 500.7313778150564, 'Generation': 724}
{'Min': -3.670007307381151, 'Max': 1014.0082487068091, 'Avg': 443.73403218727464, 'Std': 498.77222425005704, 'Generation': 725}
{'Min': -3.745683288179869, 'Max': 1012.7405239351216, 'Avg': 490.5963224709812, 'Std': 501.951421131208, '

{'Min': -3.8906070178312113, 'Max': 1013.4745737310396, 'Avg': 467.1921668561112, 'Std': 501.00881020356996, 'Generation': 784}
{'Min': -3.4775827175051357, 'Max': 1013.303251608334, 'Avg': 480.556182255307, 'Std': 501.64595973864954, 'Generation': 785}
{'Min': -3.923736992281871, 'Max': 1013.8985973273752, 'Avg': 440.52277849089745, 'Std': 498.6024018867387, 'Generation': 786}
{'Min': -3.7205151372703966, 'Max': 1013.8376878942207, 'Avg': 467.16766687826697, 'Std': 500.8538958561096, 'Generation': 787}
{'Min': -3.79314660217641, 'Max': 1012.5862314070656, 'Avg': 430.35593168846157, 'Std': 497.0145800045567, 'Generation': 788}
{'Min': -3.774047910991616, 'Max': 1015.0171517693738, 'Avg': 493.84290095497386, 'Std': 501.8646499291542, 'Generation': 789}
{'Min': -3.8405616393491684, 'Max': 1013.6902389554172, 'Avg': 433.5776379104354, 'Std': 497.47896112975644, 'Generation': 790}
{'Min': -3.9140364548416997, 'Max': 1011.965893597577, 'Avg': 396.61072940030607, 'Std': 490.7994473852392, 'G

{'Min': -3.6602620137034805, 'Max': 1012.8609796915563, 'Avg': 463.86737374210526, 'Std': 500.6689988996068, 'Generation': 849}
{'Min': -3.6330722864667386, 'Max': 1013.2663303973197, 'Avg': 460.5283550535255, 'Std': 500.47091413001465, 'Generation': 850}
{'Min': -3.8885928542434307, 'Max': 1014.3035252226767, 'Avg': 493.91976633891375, 'Std': 501.9286246962225, 'Generation': 851}
{'Min': -3.8834657633033594, 'Max': 1013.9701134893832, 'Avg': 517.2406388828573, 'Std': 501.75270801354156, 'Generation': 852}
{'Min': -3.7766806217132434, 'Max': 1012.021691050677, 'Avg': 487.1685935672294, 'Std': 501.74950428398904, 'Generation': 853}
{'Min': -3.7244296161290684, 'Max': 1010.8281085505516, 'Avg': 470.3971361074419, 'Std': 500.91191449552866, 'Generation': 854}
{'Min': -3.7945551203718075, 'Max': 1013.7531759057123, 'Avg': 494.01266555500354, 'Std': 501.96427723172263, 'Generation': 855}
{'Min': -3.548594060093975, 'Max': 1012.6377832687023, 'Avg': 490.4969909132439, 'Std': 501.774910157754

{'Min': -3.773751416359996, 'Max': 1012.7110088668135, 'Avg': 447.02109615259883, 'Std': 498.9635089532069, 'Generation': 915}
{'Min': -3.757505698877633, 'Max': 1014.6160073187376, 'Avg': 453.91506992199584, 'Std': 499.96950358227366, 'Generation': 916}
{'Min': -3.765202310460648, 'Max': 1014.3322325583407, 'Avg': 430.2022511702871, 'Std': 496.9440888882219, 'Generation': 917}
{'Min': -3.734870606777971, 'Max': 1011.8409121453208, 'Avg': 483.68513262132143, 'Std': 501.33928215075025, 'Generation': 918}
{'Min': -3.8767162485389974, 'Max': 1013.2006541480426, 'Avg': 450.2622903787704, 'Std': 499.42937887637163, 'Generation': 919}
{'Min': -3.73836160269024, 'Max': 1014.1007609217401, 'Avg': 470.43023784957194, 'Std': 501.1366496964933, 'Generation': 920}
{'Min': -3.9345465688550654, 'Max': 1013.9989538044553, 'Avg': 477.0495435989815, 'Std': 501.30354329024334, 'Generation': 921}
{'Min': -3.947947577564328, 'Max': 1015.2167735493001, 'Avg': 507.11543871912767, 'Std': 501.77544304317456, 

{'Min': -3.659354861128214, 'Max': 1013.1018667505929, 'Avg': 483.8575873279521, 'Std': 501.8039881701812, 'Generation': 980}
{'Min': -3.9245465876426913, 'Max': 1015.0501302097124, 'Avg': 504.03733957664497, 'Std': 502.0498957364328, 'Generation': 981}
{'Min': -3.366002812541335, 'Max': 1014.2961408737085, 'Avg': 457.1284005331033, 'Std': 499.944398323988, 'Generation': 982}
{'Min': -3.663866035305894, 'Max': 1014.0451861172282, 'Avg': 513.948254096084, 'Std': 501.81768617230176, 'Generation': 983}
{'Min': -3.6684140244284222, 'Max': 1014.9180654840837, 'Avg': 497.52238342786615, 'Std': 502.13115581470964, 'Generation': 984}
{'Min': -3.5339269099629003, 'Max': 1013.0466854114052, 'Avg': 483.7398340493612, 'Std': 501.6386118059013, 'Generation': 985}
{'Min': -3.890214337077701, 'Max': 1013.6267158609692, 'Avg': 423.84181334942605, 'Std': 496.3072739961466, 'Generation': 986}
{'Min': -3.8266625781822654, 'Max': 1011.9537778723653, 'Avg': 453.77537202982097, 'Std': 499.8179593439292, 'Ge